In [1]:
# Skriptas pūpsnių atributų masyvui all_beats_attr sukurimui, o taip pat pacientų padalijimui
# į dalis: train, validate, test, sukuria tų dalių SubjCode sąrašus ir įrašo juos į diską.
#  
# Atnaujintas variantas, po to, kaip padaryti pakeitimai failų varduose 2022 03 26
#
# Skriptas zive_creat_beats_attrib skirtas zive EKG įrašuose esančių pūpsnių atributų masyvui
# all_beats_attr sukurti, kad būtų galima juos panaudoti sekų formavimui:
# - atsikratome nepageidaujamų anotacijų: 'U'  ------- ///////////////////// pataisyti, neatsikratyti
# - iš anotacijų suformuojame klasių numerius
# - apskaičiuojami ir į atributus įrašomi RR intervalai: RRl, RRr. Atributų eilutėse,
#  atitinkančios pirmą ir paskutinį pūpsnį, RRl ir RRr reikšmės lygios ???????????????????????????????????????
#  sentinel = -1  - nereikia, skaičiuojama programiškai /////////////// ///////////////////////

# Pacientų kodų sąrašas SubjCodes (userNr+file_name) paimamas iš list_npy.json, kurį
# suformuoja zive_create_npy. Masyvas all_beats_attr įrašomas į failą all_beats_attr_z.csv.

# Toliau pacientai sudalijami į dalis: train, validate, test. Šioms dalims sukūriami pacientų įrašų vardų SubjCode
# sąrašai train_subjcode_lst, validate_subjcode_lst, test_subjcode_lst ir jie įrašomi į diską, kur paskui bus
# naudojami indeksų generavimui.

import pandas as pd
import numpy as np
from pathlib import Path
from icecream import ic
import json, time, sys

from zive_util_vu import runtime, read_rec_attrib, split_SubjCode, get_annotations_table, print_annotations_table


import warnings
# warnings.filterwarnings("ignore")

def print_df_list(df_list, flag1=True, flag2=True):
    if flag1:
        print(df_list[['SubjCode', 'file_name',  'N',   'S',    'V',   'U']])
    if flag2:
        sum = np.zeros(4,'int')
        # print(' '*5, 'N',   'S',   'V',  'U')
        sum[0] = df_list.iloc[:, 4].sum()
        sum[1] = df_list.iloc[:, 5].sum()
        sum[2] = df_list.iloc[:, 6].sum()
        sum[3] = df_list.iloc[:, 7].sum()
        total = sum.sum()
        str =f"\nN:{int(sum[0]):>7}  S:{(int(sum[1])):5}  V:{int(sum[2]):5}  U:{int(sum[3]):4}  total:{total:>7}" 
        print(str)

my_os=sys.platform

print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Pasiruošimas

# //////////////// NURODOMI PARAMETRAI /////////////////////////////////////////////////////

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\DI\Data\MIT&ZIVE\VU'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI/Data/MIT&ZIVE/VU'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

#  MIT2ZIVE duomenų aplankas
db_folder = 'DUOM_VU'

# Aplankas su MIT2ZIVE EKG įrašais (.npy) ir anotacijomis (.json)
rec_folder = 'records_npy'
rec_list_npy = 'list_npy.json'
train_beats_attr_fname ='all_beats_attr_z.csv'

# Kas kiek išvedamas apdorotų duomenų skaičius
show_period = 100

# Failai pūpsnių klasių formavimui
annot_grouping = {'N':'N', 'S':'S', 'V':'V', 'U':'U'}
selected_beats = {'N':0, 'S':1, 'V':2, 'U':3}

# ///////////////////////////////////////////////////////////////////////////////

# Nuoroda į MIT2ZIVE duomenų aplanką
db_path = Path(Duomenu_aplankas, db_folder)

# Nuoroda į aplanką su MIT2ZIVE EKG įrašais (.npa) ir anotacijomis (.json)
rec_dir = Path(db_path, rec_folder)

# Nuskaitome failą info_create.json ir duomenų rinkinio parametrus
file_path = Path(rec_dir,'info_create_z.json')
with open(file_path) as json_file:
    info_create = json.load(json_file)

fs = info_create['fs'] # diskretizavimo dažnumas
SubjCodes =  info_create['SubjCodes'] # pacientų įrašų sąrašas
annot_list  = info_create['annot_list'] # anotacijų sąrašas
# print(annot_list)

# Susikuriame pagalbinį anotacijų žodyną - dictionary beats_annot
nr_sequence = list(range(13))
beats_annot = dict(zip(annot_list, nr_sequence))

print("\nSkriptas skirtas zive EKG įrašuose esančių pūpsnių atributų sarašui all_beats_attr sukurti")
print("\nBendras Zive duomenų aplankas: ", Duomenu_aplankas)
print("Zive EKG įrašų aplankas: ", rec_dir)
print("EKG įrašų atributų sąrašas:", rec_list_npy)

print("Diskretizavimo dažnis: ", fs)
print("\nPacientų įrašų kodų sąrašas:\n",SubjCodes)
print("\nAnotacijų sąrašas:\n", annot_list)

pd.set_option("display.max_rows", 1000, "display.max_columns", 19)
pd.set_option('display.width', 1000)

# Nuskaitomas įrašų sąrašas, suformuojamas atitinkamas dataframe df_list
file_path = Path(rec_dir, rec_list_npy)
with open(file_path,'r', encoding='UTF-8', errors = 'ignore') as f:
    data = json.loads(f.read())
df_list = pd.json_normalize(data, record_path =['data'])
print("\ndf_list:")
print_df_list(df_list)

# Susirandame anotacijų pasiskirstymą per įrašus
annot_list = ['N', 'S', 'V', 'U']
# Susikuriame pagalbinį anotacijų žodyną - dictionary beats_annot
nr_sequence = list(range(len(annot_list)))
beats_annot = dict(zip(annot_list, nr_sequence))
# print(beats_annot)

OS in my system :  linux

Skriptas skirtas zive EKG įrašuose esančių pūpsnių atributų sarašui all_beats_attr sukurti

Bendras Zive duomenų aplankas:  /home/kesju/DI/Data/MIT&ZIVE/VU
Zive EKG įrašų aplankas:  /home/kesju/DI/Data/MIT&ZIVE/VU/DUOM_VU/records_npy
EKG įrašų atributų sąrašas: list_npy.json
Diskretizavimo dažnis:  200

Pacientų įrašų kodų sąrašas:
 [10000, 10001, 10002, 10010, 10011, 10012, 10013, 10014, 10015, 10020, 10021, 10022, 10023, 10024, 10025, 10026, 10030, 10031, 10040, 10041, 10042, 10050, 10051, 10052, 10053, 10054, 10055, 10056, 10057, 10058, 10059, 10060, 10061, 10062, 10063, 10070, 10071, 10080, 10081, 10082, 10083, 10084, 10085, 10086, 10087, 10088, 10089, 10090, 10091, 10092, 10093, 10094, 10095, 10096, 10097, 10098, 10099, 10100, 10101, 10110, 10111, 10112, 10120, 10121, 10122, 10130, 10131, 10132, 10133, 10134, 10135, 10136, 10137, 10138, 10139, 10140, 10150, 10151, 10152, 10153, 10154, 10155, 10156, 10157, 10158, 10159, 10160, 10161, 10170, 10180, 10181, 1

In [ ]:
# II-a dalis. Formuojamas freimas all_beats_attr ir  įrašomas į diską. 
# Galima praleisti ir eiti tiesiai prie III-ios dalies

print("\nEKG įrašams suformuojamas pūpsnių atributų freimas ir įrašomas į diską")

# Pacientų įrašų sąrašas bandymams:
# SubjCodes = ['10001']

# Sukūriame masyvą sekų atributų sąrašo kaupimui
all_beats_attr = pd.DataFrame({'userNr': pd.Series(dtype='int'),
                   'recordingNr': pd.Series(dtype='int'),
                   'sample': pd.Series(dtype='int'),
                   'symbol': pd.Series(dtype='str')})

start_time = time.time()

# CIKLAS PER PACIENTŲ ĮRAŠUS

# for SubjCode in SubjCodes:
for idx, row in df_list.iterrows():    
    SubjCode = row['SubjCode']
    print("\nECG įrašas:", SubjCode)
    
    # Paciento anotacijų ir EKG įrašų nuskaitymas, sekų išpjovimas ir įrašymas, sekų atributų formavimas

    # Nuskaitome paciento anotacijas ir jų indeksus
    atr_sample, atr_symbol = read_rec_attrib(rec_dir, SubjCode)

    subject_labels = []
    beat_nr = 0
    icycle = 0

    # Ciklas per visas paciento įrašo anotacijas (simbolius) ir jų vietas (i_sample)
    for i, i_sample in enumerate(atr_sample):
    
        icycle +=1
        if (icycle%show_period == 0):
            print(icycle, end =' ') 

        # Formuojame pūpsnio atributus
        userNr, recordingNr = split_SubjCode(SubjCode)

        beats_attr = {'userNr':int(userNr), 'recordingNr':int(recordingNr), 'sample':int(i_sample), 'symbol':str(atr_symbol[i])}
        
        # Senas variantas
        # train_beats_attr = train_beats_attr.append(beats_attr, ignore_index=True)

        # Variantas su concat
        df_new_row = pd.DataFrame([beats_attr])
        all_beats_attr = pd.concat([all_beats_attr, df_new_row])

        # Galima perrašyti ir kitaip, naudojant pd.DataFrame.from_records
        # all_beats_attr = pd.concat([all_beats_attr, pd.DataFrame.from_records([beats_attr])])

        beat_nr +=1

# Ciklo per pacientų įrašus pabaiga

# Atsikratome nepageidaujamų anotacijų: 'U'
# index_names = train_beats_attr[train_beats_attr['symbol'].isin(['U'])].index
# train_beats_attr.drop(index_names, inplace = True)
# print(train_beats_attr.info())

# Pernumeruojame indeksus, kad būtų nuo 0 iš eilės
all_beats_attr.reset_index(inplace = True, drop = True)

# Iš anotacijų suformuojame klasių numerius ir pridedame, kaip naują stulpelį
annot_labels = {key:selected_beats[value] for key, value in annot_grouping.items()}
labels_from_annot = all_beats_attr['symbol'].replace(annot_labels, inplace=False)
all_beats_attr['label'] = labels_from_annot

print("\n")
print("\nVisi duomenys\n")
labels_table, labels_sums = get_annotations_table(all_beats_attr)
print_annotations_table(labels_table, labels_sums, Flag1=True, Flag2=False)

# Įrašome sekos atributų masyvą į seq_dir aplanką
file_path = Path(rec_dir, train_beats_attr_fname)
all_beats_attr.index.name = "id"

all_beats_attr.to_csv(file_path, )
print("\nAtributų freimas įrašytas: į ", file_path, "\n" )

end_time = time.time()
print('\n')
runtime(end_time-start_time)

print("\nPabaiga.............")


In [4]:
#  III-a dalis. SUDALIJAME PACIENTUS IR JŲ ĮRAŠUS Į TRAIN, VALIDATION IR TEST DALIS
# Gausime df_train, df_validation, df_test, atitinkamus SubjCodes sąrašus įrašome į diską

from sklearn.model_selection import train_test_split

def split_stratified_into_train_val_test(df_input, stratify_colname='y',
                                         frac_train=0.6, frac_val=0.15, frac_test=0.25,
                                         random_state=None):
    '''
    https://localcoder.org/stratified-splitting-of-pandas-dataframe-into-training-validation-and-test-set

    Splits a Pandas dataframe into three subsets (train, val, and test)
    following fractional ratios provided by the user, where each subset is
    stratified by the values in a specific column (that is, each subset has
    the same relative frequency of the values in the column). It performs this
    splitting by running train_test_split() twice.

    Parameters
    ----------
    df_input : Pandas dataframe
        Input dataframe to be split.
    stratify_colname : str
        The name of the column that will be used for stratification. Usually
        this column would be for the label.
    frac_train : float
    frac_val   : float
    frac_test  : float
        The ratios with which the dataframe will be split into train, val, and
        test data. The values should be expressed as float fractions and should
        sum to 1.0.
    random_state : int, None, or RandomStateInstance
        Value to be passed to train_test_split().

    Returns
    -------
    df_train, df_val, df_test :
        Dataframes containing the three splits.
    '''

    if frac_train + frac_val + frac_test != 1.0:
        raise ValueError('fractions %f, %f, %f do not add up to 1.0' % \
                         (frac_train, frac_val, frac_test))

    if stratify_colname not in df_input.columns:
        raise ValueError('%s is not a column in the dataframe' % (stratify_colname))

    X = df_input # Contains all columns.
    y = df_input[[stratify_colname]] # Dataframe of just the column on which to stratify.

    # Split original dataframe into train and temp dataframes.
    df_train, df_temp, y_train, y_temp = train_test_split(X,
                                                          y,
                                                          stratify=y,
                                                          test_size=(1.0 - frac_train),
                                                          random_state=random_state)

    # Split the temp dataframe into val and test dataframes.
    relative_frac_test = frac_test / (frac_val + frac_test)
    df_val, df_test, y_val, y_test = train_test_split(df_temp,
                                                      y_temp,
                                                      stratify=y_temp,
                                                      test_size=relative_frac_test,
                                                      random_state=random_state)

    assert len(df_input) == len(df_train) + len(df_val) + len(df_test)

    return df_train, df_val, df_test


print("\nIII-a dalis. SUDALIJAME PACIENTUS IR JŲ ĮRAŠUS Į TRAIN, VALIDATION IR TEST DALIS")

# Surenkame informaciją apie pacientus

df_sum = df_list.groupby(['userId'],sort = False).sum()
# print(df_sum)
# https://sparkbyexamples.com/pandas/pandas-groupby-sum-examples/
count = df_list['userId'].value_counts()
print(f'\nViso pacientų: {len(count)}  EKG įrašų: {len(df_list)}')
print(f'\nĮrašų pasiskirstymas per pacientus')
count = count.rename("count")
frames = [df_sum, count]
result = pd.concat(frames, axis = 1)
result.index.rename ('userId', inplace= True)
# https://www.shanelynn.ie/pandas-drop-delete-dataframe-rows-columns/

print(result[['N',   'S',    'V',   'U', 'count']])
print_df_list(df_list, flag1=False, flag2=True)

# Išplečiame freimą result iki pozymio user_type, kurį naudosime kaip stratify splitinant į train, test

# Nustatome user_type pagal bendrus 'S' ir 'V' skaičius per visus paciento įrašus 
# user_type == 0, jei  bendras skaičius 'S'==0 ir bendras skaičius 'V'==0
# user_type == 1, jei  bendras skaičius 'S'> 0 ir bendras skaičius 'V'==0
# user_type == 2, jei  bendras skaičius 'S'==0 ir bendras skaičius 'V'> 0
# user_type == 3, jei  bendras skaičius 'S'> 0 ir bendras skaičius 'V'>0

result = result.reset_index()
user_types = np.zeros(len(result), dtype = int )
userIds = []
for i, row in result.iterrows():
    # print(row)
    userIds.append(row['userId'])
    if ((row['S'] == 0) & (row['V'] == 0)):
        user_types[i] = 0
    if ((row['S'] > 0) & (row['V'] == 0)):
        user_types[i] = 1
    if ((row['S'] == 0) & (row['V'] > 0)):
        user_types[i] = 2
    if ((row['S'] > 0) & (row['V'] > 0)):
        user_types[i] = 3

# print(userIds)

# df_userId = pd.DataFrame(columns = ['userId', 'user_types'])
df_userId = pd.DataFrame()
df_userId['userID'] = userIds
df_userId['user_types'] = user_types

# print("\ndf_user_Id:\n", df_userId)
# print(user_types)
# result['user_type'] = user_types
# print("\nresult\n", result)

frac_train=0.6
frac_val=0.2
frac_test=0.2
train, validation, test = split_stratified_into_train_val_test(df_userId, stratify_colname='user_types',
                                        frac_train=frac_train, frac_val=frac_val, frac_test=frac_test, random_state=2022)

print(f"frac_train: {frac_train}, frac_val: {frac_val}, frac_test: {frac_test}")
df_train = df_list.loc[df_list['userId'].isin(list(train['userID']))]
print("\ndf_train:")
print_df_list(df_train, False, True)
train_subjcode_lst = list(df_train['SubjCode'])
# print(train_subjcode_lst)
file_path = Path(rec_dir, 'train_subjcode_lst.csv')
np.savetxt(file_path, np.array(train_subjcode_lst), delimiter=',', fmt='%d')
print(f"\nMokymo imties SubjCode sąraše yra {len(train_subjcode_lst)} įrašai")
print("Mokymo imties SubjCode sąrašas įrašytas į:", file_path)

df_validation = df_list.loc[df_list['userId'].isin(list(validation['userID']))]
print("\ndf_validation:")
print_df_list(df_validation, False, True)
validation_subjcode_lst = list(df_validation['SubjCode'])
# print(validation_subjcode_lst)
file_path = Path(rec_dir, 'validation_subjcode_lst.csv')
np.savetxt(file_path, np.array(validation_subjcode_lst), delimiter=',', fmt='%d')
print(f"\nValidacinės imties SubjCode sąraše yra {len(validation_subjcode_lst)} įrašai")
print("Validacinės imties SubjCode sąrašas įrašytas į:", file_path)

df_test = df_list.loc[df_list['userId'].isin(list(test['userID']))]
print("\ndf_test:")
print_df_list(df_test, False, True)
test_subjcode_lst = list(df_test['SubjCode'])
# print(test_subjcode_lst)
file_path = Path(rec_dir, 'test_subjcode_lst.csv')
np.savetxt(file_path, np.array(test_subjcode_lst), delimiter=',', fmt='%d')
print(f"\nTestinės imties SubjCode sąraše yra {len(test_subjcode_lst)} įrašai")
print("Testinės imties SubjCode sąrašas įrašytas į:", file_path)




III-a dalis. SUDALIJAME PACIENTUS IR JŲ ĮRAŠUS Į TRAIN, VALIDATION IR TEST DALIS

Viso pacientų: 87  EKG įrašų: 457

Įrašų pasiskirstymas per pacientus
                             N     S     V  U  count
userId                                              
6034c808d6c2740008035ede  6470     7     0  0      3
60a917b354352a3df86dc1f2  9970    16    10  5      6
60e1d80f93b55b41529e9eaa  7238    11    72  0      7
613b1bc63d08d41309cdc8f1  1186     0     1  0      2
613b1c013d08d44862cdc8f2  1858     3     0  0      3
613b1c6f3d08d4370acdc8f3  5658    23     2  0     10
613b1c9c3d08d43181cdc8f4  2498     1     2  0      4
613b1cc33d08d4c0d5cdc8f5  1260     1     0  0      2
613b1d0c3d08d413ffcdc8f6  7930    26   145  0     10
613b1d673d08d4d1f3cdc8f8  7784    91     0  5     10
613b1d903d08d4df57cdc8f9  1518     1     0  0      2
613b1db23d08d4ea68cdc8fa  2105     1     4  0      3
6143507abd0cc5051b275171  1885     5     4  0      3
6144c4fbbd0cc552e427535f  6080     0  2275  0     10